<a href="https://colab.research.google.com/github/RafaelGodoyEbert/xdelta3-python/blob/master/Colab_x_delta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Link e Download

In [ ]:
##Instalar dependencias

!sudo apt-get install -y xdelta3
!pip install xdelta3
!pip install py7zr
!pip install tqdm

###

import requests
import os
import zipfile
import subprocess
from tqdm import tqdm
import py7zr
from google.colab import files
from IPython.display import clear_output

clear_output()

#@markdown ###Coloque o Link do arquivo original
url_original = 'Coloque o link do arquivo original aqui'  #@param {type:"string"}
#@markdown ###Arquivo modificado
url_modified = 'Coloque o link do arquivo modificado aqui'  #@param {type:"string"}
#@markdown ###Ative isso daqui caso queira enviar pro Google Drive, assim você consegue baixar bem mais rápido do que pelo Colab. (Mas precisa ter o espaço indicado disponivel na sua conta)
gDrive = False #@param{type:"boolean"}
#@markdown ###Baixa pelo Google Colab, demora muito mais, mas não ocupa espaço
dColab = True #@param{type:"boolean"}

def download_file(url, folder):
    filename = os.path.join(folder, os.path.basename(url))

    # Verifique se o arquivo já existe antes de iniciar o download
    if os.path.exists(filename):
        print(f"O arquivo {os.path.basename(url)} já foi baixado. Pulando o download.")
        return filename

    #print(f"Fazendo download do {os.path.basename(url)}")
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(filename, 'wb') as file, tqdm(
        desc=f"Fazendo download de {os.path.basename(filename)}",
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            file.write(data)
            bar.update(len(data))
    return filename

def extract_zip(filepath, extract_to):
    print(f"Extraindo arquivos de {os.path.basename(filepath)}")
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        total_files = len(zip_ref.namelist())
        with tqdm(total=total_files, unit='file') as pbar:
            for file in zip_ref.namelist():
                zip_ref.extract(file, extract_to)
                pbar.update(1)
    print("Arquivo ZIP extraído com sucesso!")

def extract_7z_using_7zip(filepath, extract_to, password=None):
    # Construa o comando 7-Zip com a opção de senha, se fornecida
    command = ['7z', 'x', filepath, f'-o{extract_to}']
    if password:
        command.extend(['-p' + password])

    print(f"Extraindo arquivos de {os.path.basename(filepath)}")
    # Execute o comando 7-Zip e crie uma barra de progresso
    with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, universal_newlines=True) as process:
        total_size = None
        for line in process.stdout:
            if "Compressing" in line:
                parts = line.split()
                if len(parts) > 2 and parts[0].isdigit() and parts[2].isdigit():
                    total_size = int(parts[2])

            if total_size is not None:
                # Atualiza a barra de progresso com base no número de bytes extraídos
                parts = line.split()
                if len(parts) > 1 and parts[0].isdigit():
                    bytes_extracted = int(parts[0])
                    with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024) as bar:
                        bar.update(bytes_extracted)
        print(f"Arquivos de {os.path.basename(filepath)} extraídos com sucesso!")

    # Execute o comando 7-Zip
    subprocess.run(command)

def apply_xdelta(original_file, xdelta_file, output_file):
    subprocess.run(["xdelta3", "-f", "-d", "-s", original_file, xdelta_file, output_file])

def main():
    original_folder = 'original_folder'
    modified_folder = 'modified_folder'
    patch_file = 'patch_file'
    patched_file = 'patched_file'

    if not os.path.exists(original_folder):
        os.makedirs(original_folder)

    if not os.path.exists(modified_folder):
        os.makedirs(modified_folder)

    original_path = download_file(url_original, original_folder)
    modified_path = download_file(url_modified, modified_folder)

    # Verifique se os arquivos são ZIP e extraia-os, se necessário
    if original_path.lower().endswith(".zip"):
        extract_zip(original_path, original_folder)
        original_path = os.path.join(original_folder, os.listdir(original_folder)[0])
    #print("Arquivo ZIP extraído com sucesso!")

    if modified_path.lower().endswith(".zip"):
        extract_zip(modified_path, modified_folder)
    elif modified_path.lower().endswith(".7z"):
        password = input("Digite a senha do arquivo 7z (se necessário): ")
        extract_7z_using_7zip(modified_path, modified_folder, password=password)
    #print("Arquivo 7z extraído com sucesso!")

    # Procure o arquivo .xdelta dentro das pastas extraídas do arquivo modificado
    for root, _, files in os.walk(modified_folder):
        for file in files:
            if file.lower().endswith(".xdelta"):
                xdelta_path = os.path.join(root, file)
                break

    # Aplica o arquivo .xdelta no arquivo original
    apply_xdelta(original_path, xdelta_path, patched_file)

    print(f"Patch aplicado com sucesso! O arquivo patched está pronto: {patched_file}")

if __name__ == "__main__":
    main()

# Substitua 'caminho_da_pasta' pelo caminho da pasta onde o arquivo está localizado
caminho_da_pasta = 'original_folder'

# Lista os arquivos na pasta
arquivos = os.listdir(caminho_da_pasta)

# Obtém o nome e a extensão do primeiro arquivo na lista
nome_do_arquivo, extensao_do_arquivo = os.path.splitext(arquivos[0])

print(f'O arquivo {nome_do_arquivo} tem a extensão {extensao_do_arquivo}')

# Renomear o arquivo patched com a extensão do arquivo modificado
os.rename('patched_file', f'patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}')

clear_output()

if gDrive:
  from google.colab import drive
  drive.mount('/content/drive')

  import shutil

  caminho_do_arquivo = f'/content/patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}'
  diretorio_de_destino = f'/content/drive/MyDrive/patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}'

  clear_output()

  # Move o arquivo para a pasta content/drive
  shutil.copy(caminho_do_arquivo, diretorio_de_destino)
  print('O patch foi aplicado, pode ir no seu Drive baixar o arquivo!')
  print('Às vezes demora alguns minutos para atualizar no seu Drive')
  print(' ')
  print(' ')
  print(f'Nome do arquivo é patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}')

if dColab:
  files.download(f'/content/patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}')
  print('O arquivo só começa a baixar após a barra ficar cheia')
  print('O tempo de demora é conforme o peso do arquivo')
  print(' ')
  print(' ')
  print(f'Nome do arquivo é patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}')

#Caso mude de opinião, você ainda pode enviar pro GoogleDrive com essa opção

In [ ]:
#@markdown #Enviar pro Google Drive

from google.colab import drive
drive.mount('/content/drive')

import shutil

caminho_do_arquivo = f'/content/patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}'
diretorio_de_destino = f'/content/drive/MyDrive/patched_file(Godoyy)-{nome_do_arquivo}{extensao_do_arquivo}'

# Move o arquivo para a pasta content/drive
shutil.copy(caminho_do_arquivo, diretorio_de_destino)
  print('O patch foi aplicado, pode ir no seu Drive baixar o arquivo!')
  print('As vezes demora alguns minutos para atualizar no seu Drive')

# Créditos / Contato

Rafael Godoy<br>
[Github](https://github.com/RafaelGodoyEbert)<br>
[Instagram](https://www.instagram.com/rafael.godoy.ebert/)<br>
[Twitter / X](https://twitter.com/GodoyEbert)
<br>
[Discord](https://discord.com/invite/atFQmmR2fy
)
<br>

# Agradecimentos
Samuelcolvin<br>
[Github](https://github.com/samuelcolvin/xdelta3-python)<br>